Gabriel Avinaz\
Week 7\
4/26/23

## Assignment 7.1
### Part a

In [248]:
import os
import shutil
import pandas as pd

In [249]:
path = "data"
if not os.path.exists(path):
    os.makedirs(path)
    shutil.copy("../assignment03/results/routes.parquet", path)

In [250]:
routes_df = pd.read_parquet("data/routes.parquet", engine='pyarrow')
routes_df = pd.json_normalize(routes_df["Flight Info"])
routes_df.sample(10)

codeshare   equipment  airline.active  airline.airline_id  \
2983       False       [CNA]            True                1109   
37778      False       [763]            True                3200   
59733      False       [757]            True                5265   
29127      False       [M80]            True                  35   
34356      False       [320]            True                4867   
17958      False  [320, 321]            True                1680   
57615       True  [333, 343]            True                5209   
61559      False  [321, AT7]            True                5309   
23815      False       [DH8]            True                2220   
42288       True       [73H]            True                1758   

                     airline.alias   airline.callsign         airline.country  \
2983   Pulkovo Aviation Enterprise                nan      Russian Federation   
37778                          nan                LAN                   Chile   
59733                          TWA            U S AIR           United States   
29127                           \N          ALLEGIANT           United States   
34356                          nan                TAM                  Brazil   
17958         SN Brussels Airlines             CATHAY  Hong Kong SAR of China   
57615                          TWA             UNITED           United States   
61559                          TWA  VIET NAM AIRLINES                 Vietnam   
23815            Emirates Airlines          ETHIOPIAN                Ethiopia   
42288         SN Brussels Airlines      CHINA EASTERN                   China   

      airline.iata airline.icao            airline.name  ...  \
2983            8D          nan                  Astair  ...   
37778           LA          LAN            LAN Airlines  ...   
59733           US          USA              US Airways  ...   
29127           G4          AAY           Allegiant Air  ...   
34356           JJ          TAM  TAM Brazilian Airlines  ...   
17958           CX          CPA          Cathay Pacific  ...   
57615           UA          UAL         United Airlines  ...   
61559           VN          HVN        Vietnam Airlines  ...   
23815           ET          ETH      Ethiopian Airlines  ...   
42288           MU          CES  China Eastern Airlines  ...   

       src_airport.icao  src_airport.latitude src_airport.longitude  \
2983                NaN                   NaN                   NaN   
37778              SCEL            -33.393002            -70.785797   
59733              KMIA             25.793200            -80.290604   
29127              KPIE             27.910200            -82.687401   
34356              SBLO            -23.333599            -51.130100   
17958              VTSP              8.113200             98.316902   
57615              KSEA             47.449001           -122.308998   
61559              VVTS             10.818800            106.652000   
23815              FVHA            -17.931801             31.092800   
42288              YSSY            -33.946098            151.177002   

                                        src_airport.name src_airport.source  \
2983                                                 NaN                NaN   
37778  Comodoro Arturo Merino Benítez International A...        OurAirports   
59733                        Miami International Airport        OurAirports   
29127     St Petersburg Clearwater International Airport        OurAirports   
34356                      Governador José Richa Airport        OurAirports   
17958                       Phuket International Airport        OurAirports   
57615               Seattle Tacoma International Airport        OurAirports   
61559                 Tan Son Nhat International Airport        OurAirports   
23815        Robert Gabriel Mugabe International Airport        OurAirports   
42288       Sydney Kingsford Smith International Airport        OurAirports   

      src

In [251]:
routes_df = routes_df.dropna(subset=['src_airport.iata', 'dst_airport.iata', 'airline.iata'])
routes_df.isna().sum()

codeshare                     0
equipment                     0
airline.active                0
airline.airline_id            0
airline.alias                 0
airline.callsign              0
airline.country               0
airline.iata                  0
airline.icao                  0
airline.name                  0
dst_airport.airport_id        0
dst_airport.altitude          0
dst_airport.city              0
dst_airport.country           0
dst_airport.dst               0
dst_airport.iata              0
dst_airport.icao              0
dst_airport.latitude          0
dst_airport.longitude         0
dst_airport.name              0
dst_airport.source            0
dst_airport.timezone          0
dst_airport.type              0
dst_airport.tz_id             0
src_airport.airport_id        0
src_airport.altitude          0
src_airport.city              0
src_airport.country           0
src_airport.dst               0
src_airport.iata              0
src_airport.icao              0
src_airp

In [252]:
def key_gen(row):
    return str(row['src_airport.iata']) + str(row['dst_airport.iata']) + str(row['airline.iata'])

In [253]:
routes_df["key"] = routes_df.apply(key_gen, axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key']].sample(10)

src_airport.iata dst_airport.iata airline.iata       key
56751              IAH              GCM           UA  IAHGCMUA
17248              YNT              CTU           CA  YNTCTUCA
61718              CUN              ATL           VS  CUNATLVS
64244              PDX              LAS           WN  PDXLASWN
28531              BEL              GIG           G3  BELGIGG3
56343              EWR              LIR           UA  EWRLIRUA
14182              ORH              MCO           B6  ORHMCOB6
31114              BLE              GOT            -   BLEGOT-
62942              PRG              FCO           W6  PRGFCOW6
13290              FCO              CTA           AZ  FCOCTAAZ

In [254]:
partitions = (
    ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
    ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
    ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
    ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

def kv_key_gen(row):
    for tuple in partitions:
        first_char = row["key"][:1]
        if first_char >= tuple[0] and first_char <= tuple[1]:
            if tuple[0] == tuple[1]:
                return tuple[0]
            else:
                return tuple[0] + '-' + tuple[1]

In [255]:
routes_df["kv_key"] = routes_df.apply(kv_key_gen, axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key', 'kv_key']].sample(10)

src_airport.iata dst_airport.iata airline.iata       key kv_key
59337              HNL              DFW           US  HNLDFWUS    G-H
45511              SAW              ARN           PC  SAWARNPC    S-T
41872              LYA              SHA           MU  LYASHAMU    K-L
10488              GOI              PNQ           AI  GOIPNQAI    G-H
5706               LAX              GDL           AA  LAXGDLAA    K-L
52484              ISL              BSR           TK  ISLBSRTK    I-J
3434               HRB              HIA           9C  HRBHIA9C    G-H
3299               REL              BRC           8R  RELBRC8R    Q-R
8659               GVR              PLU           AD  GVRPLUAD    G-H
52220              TEB              ACK           TJ  TEBACKTJ    S-T

In [256]:
routes_df.to_parquet('results/kv/', partition_cols=['kv_key'])

### Part b

In [257]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

In [258]:
routes_df["hashed"] = routes_df.apply(lambda x: hash_key(x["key"]), axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key', 'hashed']].sample(10)

src_airport.iata dst_airport.iata airline.iata       key  \
7480               NAP              MUC           AB  NAPMUCAB   
18391              CTU              XIY           CZ  CTUXIYCZ   
34776              NRT              YVR           JL  NRTYVRJL   
57789              SYR              IAD           UA  SYRIADUA   
60244              PVD              CLT           US  PVDCLTUS   
1459               HAM              PRG           4U  HAMPRG4U   
16260              DCY              CTU           CA  DCYCTUCA   
66171              LGW              TIV           YM  LGWTIVYM   
43028              KIX              TPE           NH  KIXTPENH   
52815              SGN              BKK           TK  SGNBKKTK   

                                                  hashed  
7480   0f0aa30a5bef0043fbc05190189cd5194fc4561bc4298e...  
18391  2e8655e3ec7d850e068860f126a9b408313f69d3ff4a5b...  
34776  11606bcab419ea963ca19d37921d592c9f15398142dc8a...  
57789  e8e4d8205fefaa1c024f9f7d0ec24044c90c4c1ec2254a...  
60244  d07d8e922166f2bd1bffcea2c36cc642fd7aec84ad4888...  
1459   e30b33249a42fdaa7d004453b7f7384561890fe2ffabc0...  
16260  ed3e86e0f5e7a3e00303088d66f1bd8fc31452147a9aa7...  
66171  c7ad2355946713154cb7e71e40526c7da26bb379623d38...  
43028  eefad32e57931de45a8c6f67127ffd9556afe1aeaf2c33...  
52815  c19c4d4d9c78fa846635174770e9730b72bbc612a0d497...

In [259]:
routes_df["hash_key"] = routes_df.apply(lambda x: x["hashed"][0].upper(), axis=1)

In [260]:
routes_df.to_parquet('results/hash/', partition_cols=['hash_key'])

### Part c

In [261]:
import pygeohash

In [262]:
data_centers = {'Location': ['West', 'Central', 'East'],
        'City': ['The Dalles, Oregon', 'Papillion, NE', 'Loudoun County, Virginia'],
        'Latitude': [45.5945645, 41.1544433, 39.08344],
        'Longitude': [-121.1786823, -96.0422378, -77.6497145]}

data_center_df = pd.DataFrame(data_centers)
data_center_df


Location                      City   Latitude   Longitude
0     West        The Dalles, Oregon  45.594564 -121.178682
1  Central             Papillion, NE  41.154443  -96.042238
2     East  Loudoun County, Virginia  39.083440  -77.649715

In [263]:
# Assignment 7.1.c
routes_df['src_airport_geohash'] = routes_df.apply(
    lambda row: pygeohash.encode(row['src_airport.latitude'], row['src_airport.longitude']), axis=1
)
def determine_location(src_airport_geohash):
    locations = dict(
        central=pygeohash.encode(41.1544433, -96.0422378),
        east=pygeohash.encode(39.083440, -77.649715),
        west=pygeohash.encode(45.594564, -121.178682)
        ## TODO: add west and east
    )
    
    distances = [[pygeohash.geohash_haversine_distance(locations[center], src_airport_geohash), center] for center in locations]
    
    distances.sort()
    return distances[0][1]
routes_df['location'] = routes_df['src_airport_geohash'].apply(determine_location)
routes_df.to_parquet('results/geo', partition_cols=['location'])

In [264]:
routes_df[['src_airport.iata', 'src_airport.latitude', 'src_airport.longitude', 'key', 'location']].sample(10)

src_airport.iata  src_airport.latitude  src_airport.longitude       key  \
220                CAC            -25.000299             -53.500801  CACGRU2Z   
37896              LUX             49.623333               6.204444  LUXAYTLG   
4342               GVA             46.238098               6.108950  GVAOPOA3   
1741               KWN             59.755100            -161.845000  KWNEEK4Y   
24736              FRA             50.033333               8.570556  FRAKEFFI   
29447              LRR             27.674700              54.383301  LRRSHJG9   
11688              KTN             55.355598            -131.714004  KTNSITAS   
13427              JFK             40.639801             -73.778900  JFKYYZAZ   
5834               LHR             51.470600              -0.461941  LHREDIAA   
67240              SZX             22.639299             113.810997  SZXWUHZH   

      location  
220       east  
37896     east  
4342      east  
1741      west  
24736     east  
29447     east  
11688     west  
13427     east  
5834      east  
67240     west

In [265]:
routes_df.to_parquet('results/geo', partition_cols=['location'])

### Part d

In [266]:
def balance_partitions(keys, num_partitions):
    partition_size = (len(keys) + num_partitions - 1) // num_partitions
    partitions = [keys[i:i + partition_size] for i in range(0, len(keys), partition_size)]
    partitions = [sorted(partition) for partition in partitions]
    return partitions

In [267]:
balanced_example = balance_partitions(list(routes_df.key), 30)
print(len(balanced_example))

for partition in balanced_example:
    print(len(partition))

print(balanced_example[0])

30
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2217
['ABJBOY2J', 'ABJOUA2J', 'ACAMEX4O', 'ACATLC4O', 'ACCDKR3G', 'ACCKMS5G', 'ACCLOS3G', 'ACCOUA2J', 'ACCROB3G', 'ACHEBA3L', 'ADQKLN2O', 'AEPMVD5Q', 'AEPPDP5Q', 'AERKZN2B', 'AGPSTR4U', 'AGPTXL4U', 'AGUMEX4O', 'AJRLYC2N', 'AKPFAI-', 'AKVYIK-', 'AKVYPX-', 'ALGCGN4U', 'AMDBLR6E', 'AMDBOM6E', 'AMDCCU6E', 'AMDDEL6E', 'AMDGOI6E', 'AMDJAI6E', 'AMDMAA6E', 'AMDPNQ6E', 'AMSHAM4U', 'AMSNDR3O', 'AMSTNG3O', 'ANFIQQ5P', 'ANUNEV5M', 'ANUPTP3S', 'AOIDUS4U', 'AOITXL4U', 'ARHCSH5N', 'ARHLED5N', 'ARHMMK5N', 'ARHNNM5N', 'ARHSVO5N', 'ARHUSK5N', 'ARNCGN4U', 'ARNDUS4U', 'ARNGEV2N', 'ARNHAD2N', 'ARNHAM4U', 'ARNJKG2N', 'ARNKRF2N', 'ARNKSD2N', 'ARNLYC2N', 'ARNMHQ2N', 'ARNOER2N', 'ARNPOR2N', 'ARNTRF2N', 'ARNVBY2N', 'ARNVHM2N', 'ARNVXO2N', 'ASFDME3R', 'ASFKZN2B', 'ASFLED3R', 'ASFMRV2B', 'ASUMVD5Q', 'ATHCGN4U', 'ATHSTR4U', 'ATLLWB3M', 'ATLMCN3M', 'ATLMEI3M', 'ATLMSL3M